In [1]:
import pandas as pd
import pdfplumber
import requests
import urllib
from bs4 import BeautifulSoup
import nltk
import re
import os
import numpy as np

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
carac_especial = ['(',')','-',',',', ',' ,','.','@',':','*','☎','●','_','n.º','º','–','nº','“','”', '%', '$','R','/','!','+','✔',
                  'A','O','Á','à','À','____','3ª','&','➧','Nº','?','1ª','','|','e','Ao', '\r','e,', '(e', '*.\r', 'É', 'J.',
                  'S/A.', 'S/A', '\t','\xa0','(DVD)', '(cd)']
all_stopwords = stopwords.words('portuguese')
char = all_stopwords
char.extend(carac_especial)
char.extend([f'{i}' for i in range(1000)])

nomes = ['esquizofrenia', 'esquizofrenia,', 'esquizofrenia.', 'esquizofrenia, ', 'esquizofrenia. ', 
         ' esquizofrenia', 'esquizofrenia ', 'esquizofrenia.\r', '"esquizofrenia"', 'esquizofrenia!', 
         'esquizofrênico','esquizofrênico,', 'esquizofrênico, ', 'esquizofrênico.', 'esquizofrênico. ',
         ', esquizofrênico']

In [32]:
def limpa_paragrafos(link, tamanho, especiais):

    html = urllib.request.urlopen(link).read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()

    r = re.compile(".*\r|\d+|.*@|.*#|.\\t|.*\\xa0|.*\\'")

    lista = []
    for i in soup.stripped_strings:
        lista.append(i)

    miolo = []
    for i in lista:
        if len(i) > tamanho:
            miolo.append(i.replace('\r\n',''))

    paragrafos = []
    for i in range(len(miolo)):
        if 'esquizo' in miolo[i]:
            paragrafos.append(miolo[i])

    for i in range(len(paragrafos)):
        lista_palavras = paragrafos[i].lower().split(' ')
        lista_palavras_sem_especiais = [word for word in lista_palavras if not word in especiais]
        especiais2 = list(filter(r.match, lista_palavras_sem_especiais))
        lista_palavras_sem_especiais2 = [word for word in lista_palavras_sem_especiais if not word in especiais2]
        limpo = ' '.join(lista_palavras_sem_especiais2)
        paragrafos[i] = limpo

    return paragrafos   


In [33]:
def unir_paragrafos(lista_links, tamanho, especiais, primeiro_1):
    indices = []
    y = []
    todos_paragrafos = []
    for indice in range(len(lista_links)):
        paragrafos = limpa_paragrafos(lista_links[indice], tamanho, especiais)
        todos_paragrafos.extend(paragrafos)
        indices.append(indice)
        
    y.extend(np.zeros(primeiro_1 - 2))
    y.extend(np.ones(len(todos_paragrafos) - (primeiro_1 - 2)))
    
    return todos_paragrafos, y, indices 

In [41]:
def salva_paragrafos(lista_links, tamanho, especiais):
    for i in range(len(lista_links)):
        paragrafos = limpa_paragrafos(lista_links[i], tamanho, especiais)
        
        os.makedirs(f'data_paragrafo/pasta{i+2}', exist_ok=True)
        for j in range(len(paragrafos)): 
            text_file = open(f'data_paragrafo/pasta{i+2}/paragrafo{j}.txt', 'wt')
            text_file.write(paragrafos[j])
            text_file.close()
        

In [35]:
def gerador_database(lista_links, tamanho, especiais, primeiro_1):
    for i in range(len(lista_links)):
        
        os.makedirs(f'pasta{i+2}')
        salva_paragrafos(lista_links, tamanho, especiais, primeiro_1, f"pasta{i+2}")

In [6]:
# link = 'https://www1.folha.uol.com.br/folha/ciencia/ult306u2066.shtml?origin=folha'
# link = 'https://www1.folha.uol.com.br/folha/ciencia/ult306u2071.shtml?origin=folha'
link ='https://www1.folha.uol.com.br/fsp/ciencia/fe1801200107.htm'
# link= 'https://www1.folha.uol.com.br/fsp/opiniao/fz0111200111.htm'

tamanho = 70
especiais = char

In [7]:
paragrafos = limpa_paragrafos(link, tamanho, especiais)
paragrafos

['folha s.paulo cérebro: pesquisa pode auxiliar tratamento esquizofrenia despersonalização',
 'pesquisa publicada hoje revista "nature". segundo coordenador, julian paul keenan, escola médica harvard, pode ajudar entender melhor tratar problemas esquizofrenia assomatopagnosia, síndrome paciente consegue identificar extremidades próprio corpo.',
 'segundo pesquisador, estudo mostra pode ser possível aumentar autopercepção paciente sofra esquizofrenia despersonalização mudando maneira lado direito cérebro estimulado.']

In [36]:
lista_links = [
    "https://www1.folha.uol.com.br/fsp/opiniao/fz1212200110.htm",
    "https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u691.shtml",
]

In [10]:
todos, y = unir_paragrafos(lista_links, tamanho, especiais, 3)
todos

['estima-se cada pessoas deve apresentar transtorno mental longo vida cerca milhões brasileiros estão, neste momento, sofrendo causa transtorno mental. tema alcançado maior importância últimas décadas, particularmente causa alta prevalência determinados transtornos mentais, alcoolismo, depressão quadros fóbico-ansiosos; gravidade outros, esquizofrenia, transtorno bipolar quadros demenciais; sobrecarga transtornos mentais representam família sociedade; desrespeito direitos fundamentais portadores desses transtornos; discriminação constantemente submetidos.',
 'folha online equilíbrio unifesp cria site disseminar informações sobre esquizofrenia',
 'familiares portadores esquizofrenia poderão trocar disseminar informações sobre doença meio site criado programa esquizofrenia departamento psiquiatria unifesp (universidade federal paulo).']

In [11]:
y

[0.0, 1.0, 1.0]

In [25]:
gerador_database(lista_links, tamanho, especiais, 3)

In [26]:
paragrafa = limpa_paragrafos(lista_links[0], tamanho, especiais)
paragrafa

['estima-se cada pessoas deve apresentar transtorno mental longo vida cerca milhões brasileiros estão, neste momento, sofrendo causa transtorno mental. tema alcançado maior importância últimas décadas, particularmente causa alta prevalência determinados transtornos mentais, alcoolismo, depressão quadros fóbico-ansiosos; gravidade outros, esquizofrenia, transtorno bipolar quadros demenciais; sobrecarga transtornos mentais representam família sociedade; desrespeito direitos fundamentais portadores desses transtornos; discriminação constantemente submetidos.']

In [40]:
salva_paragrafos(lista_links, tamanho, especiais)

In [42]:
lista_links2 = [
    "https://www1.folha.uol.com.br/fsp/ilustrad/fq0401200121.htm",
"https://www1.folha.uol.com.br/folha/mundo/ult94u16986.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1001200106.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2066.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2071.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u9192.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2087.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1801200107.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2418.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq0802200113.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2546.shtml?origin=folha",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u10685.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/folhatee/fm1902200116.htm",
"https://www1.folha.uol.com.br/fsp/folhatee/fm1902200117.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u2709.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1403200101.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u3019.shtml?origin=folha",
"https://www1.folha.uol.com.br/fsp/ciencia/fe2103200105.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs2503200110.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2903200109.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0704200102.htm",
"http://www1.folha.uol.com.br/fsp/mais/fs0804200111.htm",
"http://www1.folha.uol.com.br/folha/ciencia/ult306u3264.shtml",
"http://www1.folha.uol.com.br/fsp/ciencia/fe1104200101.htm",
"http://www1.folha.uol.com.br/folha/ciencia/ult306u3428.shtml",
"https://www1.folha.uol.com.br/fsp/mundo/ft1105200113.htm",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1305200108.htm",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1305200104.htm",
"https://www1.folha.uol.com.br/fsp/mundo/ft1705200108.htm",
"https://www1.folha.uol.com.br/folha/reuters/ult112u2519.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq0507200113.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u194.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq1207200115.htm",
"https://www1.folha.uol.com.br/folha/reuters/ult112u3660.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq2607200103.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u240.shtml",
"https://www1.folha.uol.com.br/folha/reuters/ult112u4288.shtml",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1208200128.htm",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1208200127.htm",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1208200126.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u300.shtml",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u302.shtml",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u4592.shtml",
"https://www1.folha.uol.com.br/folha/reuters/ult112u4627.shtml",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1508200104.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u4618.shtml",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u4930.shtml",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq1810200106.htm",
"https://www1.folha.uol.com.br/folha/bbc/ult272u6836.shtml",
"https://www1.folha.uol.com.br/fsp/ciencia/fe0511200101.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0711200113.htm",
"https://www1.folha.uol.com.br/folha/ciencia/ult306u5294.shtml",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1711200121.htm",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u19406.shtml",
"https://www1.folha.uol.com.br/folha/mundo/ult94u34454.shtml",
"https://www1.folha.uol.com.br/fsp/ciencia/fe1012200101.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz1212200110.htm",
"https://www1.folha.uol.com.br/folha/equilibrio/noticias/ult263u691.shtml",
"https://www1.folha.uol.com.br/fsp/cotidian/ff3012200118.htm",
"https://www1.folha.uol.com.br/folha/reuters/ult112u10188.shtml",
"https://www1.folha.uol.com.br/folha/reuters/ult112u10191.shtml",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1105200123.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc0204200112.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq3105200111.htm",
"https://www1.folha.uol.com.br/fsp/folhatee/fm0207200104.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0207200113.htm",
"https://www1.folha.uol.com.br/folha/cotidiano/ult95u33066.shtml",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0408200119.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq1708200129.htm",
"https://www1.folha.uol.com.br/fsp/acontece/ac0909200103.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs0909200117.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz2109200109.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz0110200110.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz3010200111.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz0111200111.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs0411200112.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq0203200103.htm",
"https://www1.folha.uol.com.br/fsp/ombudsma/om1803200101.htm",
"https://www1.folha.uol.com.br/fsp/opiniao/fz1501200210.htm",
"https://www1.folha.uol.com.br/fsp/mundo/ft0401200215.htm",
"https://www1.folha.uol.com.br/fsp/ombudsma/om1702200202.htm",
"https://www1.folha.uol.com.br/fsp/cotidian/ff1702200217.htm",
"https://www1.folha.uol.com.br/fsp/ilustrad/fq2702200213.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq3003200203.htm",
"http://www1.folha.uol.com.br/fsp/cotidian/ff0904200204.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz2504200210.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0405200213.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0806200201.htm",
"http://www1.folha.uol.com.br/folha/pensata/ult510u63.shtml",
"http://www1.folha.uol.com.br/fsp/opiniao/fz2406200211.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq1907200228.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz2807200201.htm",
"http://www1.folha.uol.com.br/fsp/mais/fs1808200210.htm",
"http://www1.folha.uol.com.br/fsp/dinheiro/fi1109200207.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0604200304.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0504200319.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1304200310.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2512200211.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc0901200313.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0901200304.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc2601200330.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0604200304.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0504200319.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1304200310.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2204200309.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1909200207.htm",
"http://www1.folha.uol.com.br/folha/educacao/ult305u10765.shtml",
"http://www1.folha.uol.com.br/fsp/folhatee/fm2810200209.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2512200211.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc0901200313.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0901200304.htm",
"http://www1.folha.uol.com.br/fsp/brasil/fc2601200330.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz0604200304.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq0504200319.htm",
"http://www1.folha.uol.com.br/fsp/opiniao/fz1304200310.htm",
"http://www1.folha.uol.com.br/fsp/ilustrad/fq2204200309.htm",
"https://www1.folha.uol.com.br/fsp/esporte/fk2005200314.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs0106200305.htm",
"http://www1.folha.uol.com.br/fsp/mais/fs0106200311.htm",
"https://www1.folha.uol.com.br/fsp/equilibrio/eq0506200312.htm",
"https://www1.folha.uol.com.br/folha/ilustrada/ult90u33864.shtml",
"https://www1.folha.uol.com.br/fsp/opiniao/fz1607200304.htm",
"https://www1.folha.uol.com.br/fsp/mais/fs2007200310.htm",
"https://www1.folha.uol.com.br/folha/dinheiro/ult91u70666.shtml",
"https://www1.folha.uol.com.br/folha/brasil/ult96u52200.shtml",
"https://www1.folha.uol.com.br/fsp/opiniao/fz0508200309.htm",
"https://www1.folha.uol.com.br/fsp/brasil/fc1808200317.htm",
"https://www1.folha.uol.com.br/fsp/ciencia/fe3108200304.htm"
]

In [43]:
salva_paragrafos(lista_links2, tamanho, especiais)